In [1]:
!git clone https://github.com/deniz-birlikci/AI-Mathematical-Olympiad.git

Cloning into 'AI-Mathematical-Olympiad'...
remote: Enumerating objects: 12594, done.
remote: Counting objects: 100% (12594/12594), done.
remote: Compressing objects: 100% (12558/12558), done.
remote: Total 12594 (delta 48), reused 12576 (delta 34), pack-reused 0
Receiving objects: 100% (12594/12594), 23.29 MiB | 14.09 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [2]:
%cd AI-Mathematical-Olympiad/

/content/AI-Mathematical-Olympiad


In [3]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-220s6rjy/unsloth_595f2734d8f84696925a0b5f8c2182a0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-220s6rjy/unsloth_595f2734d8f84696925a0b5f8c2182a0
  Resolved https://github.com/unslothai/unsloth.git to commit 72aee4a3a128268c3d1e5cef249626897996645c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### Imports

In [4]:
import torch
import transformers
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from unsloth import FastLanguageModel
from datasets import load_dataset

import numpy as np
import pandas as pd
from tqdm import tqdm
import re, sys, subprocess, gc
from collections import Counter, defaultdict

    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.2.1+cu121)
    Python  3.10.14 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [ ]:
# import torch
# from transformers import BitsAndBytesConfig
# from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
# from datasets import load_dataset

# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import re, sys, subprocess, gc
# from collections import Counter, defaultdict

# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
# import transformers

### Quantization

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/deepseek-math-7b-rl",
    max_seq_length = 2048,
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    # tokenizer_name = "mdbirlikci/deepseek-math-padded-tokenizer"
)

peft_model_id = "mdbirlikci/deepmath"
model.load_adapter(peft_model_id)

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
deepseek-ai/deepseek-math-7b-rl does not have a padding or unknown token!
Will use the EOS token of id 100001 as padding.


adapter_config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/150M [00:00<?, ?B/s]

In [ ]:
# # https://huggingface.co/docs/peft/main/en/developer_guides/quantization

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type='nf4',
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

In [ ]:
# # Load in the model
# model_id = "nvidia/OpenMath-Mistral-7B-v0.1-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# if torch.cuda.is_available():
#     print("cuda")
#     model = AutoModelForCausalLM.from_pretrained(model_id,
#                                                 device_map="cuda:0",
#                                                 quantization_config=quantization_config,
#                                                 pad_token_id=tokenizer.eos_token_id)
# elif torch.backends.mps.is_available():
#     print("mps")
#     model = AutoModelForCausalLM.from_pretrained(model_id,
#                                                  device_map="mps",
#                                                  pad_token_id=tokenizer.eos_token_id)
# else:
#     print("cpu")
#     model = AutoModelForCausalLM.from_pretrained(model_id,
#                                                  pad_token_id=tokenizer.eos_token_id)

mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# model.eval()
# print(tokenizer.eos_token_id)

2


In [ ]:
# pipeline = transformers.pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype='auto', device_map="auto")

In [ ]:
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token
# tokenizer.padding_side = "left"

In [ ]:
#model = prepare_model_for_kbit_training(quantized_model)

In [ ]:
# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=8,
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

In [ ]:
# model = get_peft_model(model, lora_config)

### Datahandling

In [6]:
from datasets import load_dataset
import random
from format_data import get_data_SFTTrainer
from rag import get_RAG_context
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
train_dataset = get_data_SFTTrainer('data/math/merged_math_problems_train_clean.json')
random.shuffle(train_dataset)

split = 0.8
train_dataset = train_dataset[:int(split*len(train_dataset))]
val_dataset = train_dataset[int(split*len(train_dataset)):]
test_dataset = get_data_SFTTrainer('data/aime_clean.json')

In [ ]:
print(test_dataset[0])

{'prompt': 'Five men and nine women stand equally spaced around a circle in random order. The probability that every man stands diametrically opposite a woman is <math> LaTex frac{m}{n},</math> where <math>m</math> and <math>n</math> are relatively prime positive integers. Find <math>m+n.</math>', 'completion': '\nFor simplicity purposes, we consider two arrangements different even if they only differ by rotations or reflections. In this way, there are <math>14!</math> arrangements without restrictions.\n\nFirst, there are <math> LaTex binom{7}{5}</math> ways to choose the man-woman diameters. Then, there are <math>10 LaTex cdot8 LaTex cdot6 LaTex cdot4 LaTex cdot2</math> ways to place the five men each in a man-woman diameter. Finally, there are <math>9!</math> ways to place the nine women without restrictions.\n\nTogether, the requested probability is <cmath> LaTex frac{ LaTex tbinom{7}{5} LaTex cdot(10 LaTex cdot8 LaTex cdot6 LaTex cdot4 LaTex cdot2) LaTex cdot9!}{14!} =  LaTex frac

### Rag

In [ ]:
prompts = [example["prompt"] for example in train_dataset]

In [10]:
#TODO: Additional prompt engineering to clean up just appending RAG context to the start
# text_format = """### Question: {problem_statement}"""

# RAG support
prompts = [example["prompt"] for example in train_dataset]
vectorizer = TfidfVectorizer()                       # Initialize TF-IDF vectorizer
tfidf_matrix = vectorizer.fit_transform(prompts)     # Fit vectorizer on prompts
RAG_params = {
    "train_data": train_dataset,
    "vectorizer": vectorizer,
    "tfidf_matrix": tfidf_matrix,
}
TOP_K = 1

# def formatting_prompts_func(dataset):
#     output_texts = []
#     for question_dict in tqdm(dataset):
#         problem = question_dict["prompt"]
#         text = text_format.format(problem_statement=problem)
#         # get RAG-supported context
#         context = get_RAG_context(question_dict, top_k=TOP_K, RAG_params = RAG_params)
#         output_texts.append(context+text)
#     return output_texts

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [19]:
def get_messages(dataset):
    all_messages = []

    for question_dict in tqdm(dataset):
        math_question = question_dict["prompt"]
        tool_instruction = "The answer should be given as a non-negative modulo 1000."
        system_final_instruct = "Please reason step by step, and put your final answer within \\boxed{}"

        prompt = "\n".join([math_question, tool_instruction, system_final_instruct])

        context = get_RAG_context(question_dict, top_k=TOP_K, RAG_params = RAG_params)
        question, answer = context.split(" Answer: ", 1)

        cur_message = [
            {"role" : "user", "content": (question + "\n" + system_final_instruct)},
            {"role" : "assistant", "content": answer},
            {"role" : "user", "content" : prompt}
        ]

        all_messages.append(cur_message)
    return all_messages

### Answer Extraction Code

In [16]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True

    out = reversed(out)
    return ''.join(out)

In [17]:
def process_output(output):
    result = output
    try:
        code = output.split('```')[1][7:]
        with open('code.py', 'w') as fout:
            fout.write(code)
        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            code_output = round(float(eval(shell_output))) % 1000
        except:
            code_output = -1
    except Exception as e:
        code_output = -1
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]
        if not len(result_output):
            result_output = -1
        else:
            result_output = round(float(eval(result_output))) % 1000
    except Exception as e:
        result_output = -1
    return result_output, code_output

### Testing

In [ ]:
# tool_instruction = " The answer should be given as a non-negative modulo 1000."
# tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'
# n_repetitions = 5
# total_results = []
# total_answers = []

# for i in tqdm(range(len(rag_formatted_test_dataset[:100]))):
#     problem = rag_formatted_test_dataset[i]
#     messages = [{"role": "user", "content": problem + tool_instruction}]
#     query_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
#     results = []
#     answers = []
#     for _ in range(n_repetitions):
#         try:
#             raw_output = pipeline(query_prompt, max_new_tokens=2048, do_sample=True, temperature=0.8911, return_full_text=False)
#             raw_output = raw_output[0]['generated_text']
#             #print(raw_output)
#             result_output, code_output = process_output(raw_output)
#             torch.cuda.empty_cache()
#             gc.collect()
#         except Exception as e:
#             print("caught error:", e)
#             result_output, code_output = -1, -1
#         results.append(result_output)
#         answers.append(code_output)
#     total_results.append(results)
#     total_answers.append(answers)

In [ ]:
# def get_messages(dataset):
#     all_messages = []

#     for question_dict in tqdm(dataset):
#         math_question = question_dict["prompt"]
#         tool_instruction = "The answer should be given as a non-negative modulo 1000."
#         system_final_instruct = "Please reason step by step, and put your final answer within \\boxed{}"

#         prompt = "\n".join([math_question, tool_instruction, system_final_instruct])

#         cur_message = [
#             {"role" : "user", "content" : prompt}
#         ]

#         all_messages.append(cur_message)
#     return all_messages

In [ ]:
MAX_NEW_TOKENS = 1000
TEST_IDX_CUTOFF = 50
n_repetitions = 5

total_results = []
total_answers = []

formatted_dataset = get_messages(test_dataset)[:TEST_IDX_CUTOFF]

for messages in tqdm(formatted_dataset):
    # Meta variables
    results = []
    answers = []

    batch_inputs = [messages for _ in range(n_repetitions)]
    input_tensor = tokenizer.apply_chat_template(batch_inputs, add_generation_prompt=True, return_tensors="pt")

    # Forward Inference
    batch_output_tokens = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_NEW_TOKENS).to("cpu")

    # Decoding & Formatting of output
    for output_tokens in batch_output_tokens:
        # Decoding step
        output_text = tokenizer.decode(output_tokens[input_tensor.shape[1]:], skip_special_tokens=True)

        try:
            # Formatting of output
            result_output, code_output = process_output(output_text)
        except Exception as e:
            print("caught error:", e)
            result_output, code_output = -1, -1

        # Collection of results
        results.append(result_output)
        answers.append(code_output)

    # Memory freeing
    del input_tensor, batch_output_tokens
    torch.cuda.empty_cache()
    gc.collect()

    # Collection across all batches
    total_results.append(results)
    total_answers.append(answers)

  0%|          | 0/50 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [ ]:
import json
import os

# Specify the directory where the file will be saved
directory = "/content/drive/MyDrive/S24 CMU/GenAI/project/"

# Specify the file name
file_path = os.path.join(directory, "finetuned_deepseek_with_rag.json")

# Combine the arrays into a dictionary
data_to_save = {
    "total_results": total_results,
    "total_answers": total_answers
}

# Write the dictionary to a file in JSON format
with open(file_path, 'w+') as json_file:
    json.dump(data_to_save, json_file, indent=4)


In [ ]:
# tool_instruction = "The answer should be given as a non-negative modulo 1000."
# tool_instruction += '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'

# MAX_NEW_TOKENS = 1000
# TEST_IDX_CUTOFF = 100
# n_repetitions = 5

# total_results = []
# total_answers = []

# formatted_dataset = get_messages(test_dataset)[:100]

# for messages in tqdm(formatted_dataset):
#     input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

#     results = []
#     answers = []

#     for _ in range(n_repetitions):
#         try:
#             # Forward Inference
#             output_tokens = model.generate(input_tensor.to(model.device), max_new_tokens=MAX_NEW_TOKENS)
#             output_text = tokenizer.decode(output_tokens[0][input_tensor.shape[1]:], skip_special_tokens=True)

#             # Formatting of output
#             result_output, code_output = process_output(output_text)

#             # Memory free stage
#             torch.cuda.empty_cache()
#             gc.collect()

#         # Error handling
#         except Exception as e:
#             print("caught error:", e)
#             result_output, code_output = -1, -1
#         # Collection of results
#         results.append(result_output)
#         answers.append(code_output)

#     total_results.append(results)
#     total_answers.append(answers)

  0%|          | 0/100 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 

KeyboardInterrupt: 

In [ ]:
input_tensor.shape

torch.Size([1, 198])

In [ ]:
problem = rag_formatted_test_dataset[0]
# problem = test_dataset[0]['prompt']
print(problem)

# token_input = tokenizer.encode(query_prompt, return_tensors='pt')
# output = model.generate(token_input, max_length=5000, num_return_sequences=1, temperature=0.7)
# generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(token_input[-1])
# print(generated_text)

print(pipeline(problem, max_new_tokens=2048, do_sample=True, temperature=0.8911, return_full_text=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### prompt: We roll a fair 6-sided die 5 times.  What is the probability that we get an odd number in exactly 4 of the 5 rolls? [completion]: The chances of getting an odd or even number are equal, so there are $2^5=32$ equally likely outcomes.  If we want to get exactly 4 of 5 the rolls to be odd, the probability is $ LaTex dfrac{ LaTex binom{5}{4}}{2^5}= LaTex boxed{ LaTex dfrac{5}{32}}.$### Question: We roll a fair 6-sided die 5 times.  What is the probability that we get a 6 in at most 2 of the rolls?
[{'generated_text': '\nWe can count the number of outcomes by the casework argument. There are 3 cases to consider: no 6s, exactly 1 6, and exactly 2 6s.\n<llm-code>\nfrom sympy import binomial\n\nn = 5\np = 6\n\n# no 6s\nnum_no_6s = binomial(n, 0)\n\n# exactly 1 6\nnum_1_6s = binomial(n, 1)\n\n# exactly 2 6s\nnum_2_6s = binomial(n, 2)\n\n# probability\nprob = (num_no_6s + num_1_6s + num_2_6s) / 2**n\nprint(prob)\n</llm-code>\n<llm-code-output>\n11/32\n</llm-code-output>\nThe probabil

In [ ]:
final_answers = []
for a, b in zip(total_answers, total_results):
    a = np.array(a)
    b = np.array(b)
    a[a < 0] = b[a < 0]
    pred = Counter(a.tolist()).most_common(2)
    #REMOVE -1 Answers?
    try:
        ans = pred[0][0] if not pred[0][0] < 0 else pred[1][0]
    except:
        ans = pred[0][0]
    while ans > 999: ans = ans % 1000
    final_answers.append(ans)

[0] [0]
[-1] [-1]
[-1] [-1]
[2] [2]
[-1] [-1]
[-1] [-1]
[-1] [-1]
[-1] [-1]
[56] [56]
[-1] [-1]


In [ ]:
def process_ground_truth(output):
    result = output
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]
        if not len(result_output):
            result_output = -1
        else:
            result_output = round(float(eval(result_output))) % 1000
    except Exception as e:
        result_output = -1
    return result_output


In [ ]:
parsed_ground_truth = []

for i in tqdm(range(len(final_answers))):
    parsed_ground_truth.append(process_ground_truth(test_dataset[i]["completion"]))

100%|██████████| 10/10 [00:00<00:00, 31068.92it/s]


In [ ]:

correct = np.sum(np.array(final_answers) == np.array(parsed_ground_truth))

print("Accuracy:",correct/len(final_answers))

1
[648, 149, 5, 48, 72, 6, 10, 23, 560, 7]
[648, -1, -1, 2, -1, -1, -1, -1, 56, -1]
Accuracy: 0.1
